In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from numpy import nan

In [2]:
# funcao que busca o produto no site do mercado livre para entregar a ordem para o cliente
def lista_vendedores_produto(nome_produto, num_paginas=3):
    pagina_pesquisa = ["", "_Desde_49", "_Desde_97", "_Desde_145", "_Desde_193", "_Desde_241", "_Desde_289", "_Desde_337", "_Desde_385", "_Desde_433"]
    
    lista_lojas = []
    for i in range(num_paginas):
        url_inicial = 'https://lista.mercadolivre.com.br/' + nome_produto + pagina_pesquisa[i]
        html_content = requests.get(url_inicial).text
        soup = BeautifulSoup(html_content, "html")

        url_list = []
        for ul in soup.find_all("ol", {"id": "searchResults"}):
            for li in ul.find_all('li'):
                a = li.find('a')
                if a != None:
                   url_list.append(a['href'])

        #remove duplciates        
        url_list = url_list[1::2]

        for link in url_list:
            html_content = requests.get(link).text
            soup = BeautifulSoup(html_content, "html")
            a = soup.find("a", {"id": "seller-view-more-link"})
            
            if a != None:
                html_content = requests.get(a["href"]).text
                soup = BeautifulSoup(html_content, "html")
                h3= soup.find("h3", {"id": "store-info__name"})
                if h3 != None:
                    lista_lojas.append(h3.text)
                    
    lista_lojas = list(dict.fromkeys(lista_lojas))

    return lista_lojas


In [3]:
# identifica a ordem do cliente na lista de produtos
def reload(store_name, product):
    lista_lojas = lista_vendedores_produto(product)
    if store_name in lista_lojas:
        n_index = lista_lojas.index(store_name)
        n_index = n_index + 1 if n_index > 0 else nan
    else:
        n_index = nan

    if n_index > 0:
        print(f'[SUCCESS] Your index domain on MercadoLivre is : {n_index}')
    else:
        print('[ERROR] Domain not found on MercadoLivre ')

In [4]:
# teste de do cliente SARCOMPY para o produto mp3
reload('SARCOMPY', 'mp3')

[SUCCESS] Your index domain on MercadoLivre is : 4


In [5]:
# teste de do cliente LOJA DO ZEZIN para o produto mp3
reload('LOJA DO ZEZIN', 'mp3')

[ERROR] Domain not found on MercadoLivre 
